In [1]:
!nvidia-smi

Mon Sep 18 09:40:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off

In [3]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline, AutoModelForCausalLM

**Data**

In [4]:
questions_dir = Path("skyscanner")
questions_dir.mkdir(exist_ok=True, parents=True)

def write_file(question, answer, file_path):
  text = f"""
  Q: {question}
  A: {answer}
  """.strip()
  with Path(questions_dir / file_path).open("w") as text_file:
    text_file.write(text)

In [5]:
write_file(
    question="How do I search for flights on skyscanner?",
    answer="""Skyscanner helps you find the best options for flights on a specific data, or an any day in a given month or even year. For tips on how bext to search, please head over to over to our search tip
    If you're looking for inspiration for your next trip, why not try our everywhere, features. Or, if you want to hang out and ensure best price, you can set up price alers to let
    """.strip(),
    file_path="question_1.txt",
)

In [6]:
write_file(
    question="What are mash-ups",
    answer="""A mash-up is a creative work, usually a song, created by blending two or more pre-recorded songs, typically by superimposing the vocal track of one song seamlessly over the instrumental track of another and changing the tempo and key where necessary.

Mash-ups have been around for many years, but they became more popular in the late 1990s and early 2000s with the rise of digital music production software. Today, mash-ups are created by people of all skill levels, from professional musicians to amateur DJs.

Mash-ups can be created for a variety of reasons. Some people create them simply for fun, while others use them as a way to express their creativity or to make a social or political statement. Mash-ups can also be used as a tool for education, such as when teachers create mash-ups to teach their students about different genres of music or historical events.
    """.strip(),
    file_path="question_2.txt",
)

In [7]:
write_file(
    question="Why have I been blocked from accessing the Skyscanner website?",
    answer="""
There are a few reasons why you might have been blocked from accessing the Skyscanner website:

You're using a VPN. Skyscanner blocks some VPNs because they are used by bots to scrape their website.
You're using the website at super speed. If you're making too many requests to the Skyscanner website in a short period of time, they may block you to prevent their servers from being overloaded.
You're using a browser plug-in that is interfering with the website. Some browser plug-ins, such as ad blockers and privacy-focused extensions, can interfere with how Skyscanner works.
You're using an automated browser. Skyscanner blocks automated browsers because they are often used by bots to scrape their website.
If you're not sure why you've been blocked, you can try contacting Skyscanner support for assistance.
    """.strip(),
    file_path="question_3.txt",
)

In [8]:
write_file(
    question="Where is my booking confirmation?",
    answer="""Skyscanner is a travel search engine, so they don't actually book your travel for you. Instead, they redirect you to the website of the airline or travel agent that you choose to book with.

This means that your booking confirmation email will come from the airline or travel agent that you booked with, not from Skyscanner.

If you can't find your booking confirmation email, you can try the following:

Check your spam or junk mail folder.
Search your inbox for the name of the airline or travel agent that you booked with.
Try contacting the airline or travel agent directly to ask for a copy of your booking confirmation.
    """.strip(),
    file_path="question_4.txt",
)

In [9]:
write_file(
    question="How do I change or cancel my booking?",
    answer="""
To change or cancel your booking, you will need to contact the airline or travel agent that you booked with. Skyscanner does not have access to your booking information or the ability to make changes or cancellations on your behalf.

To contact the airline or travel agent, you can usually find their contact information on their website or in your booking confirmation email.

When you contact the airline or travel agent, be sure to have your booking information ready, such as your booking reference number or ticket number. This will help them to quickly locate your booking and make the necessary changes or cancellations.

Keep in mind that there may be fees associated with changing or canceling your booking. These fees vary depending on the airline or travel agent and the type of fare that you booked.
    """.strip(),
    file_path="question_5.txt",
)

In [10]:
write_file(
    question="I booked the wrong dates / times?",
    answer="""
If you booked the wrong dates or times for your flight, you will need to contact the airline or travel agent that you booked with to make changes to your booking. Skyscanner does not have access to your booking information or the ability to make changes on your behalf.

To contact the airline or travel agent, you can usually find their contact information on their website or in your booking confirmation email.

When you contact the airline or travel agent, be sure to have your booking information ready, such as your booking reference number or ticket number. This will help them to quickly locate your booking and make the necessary changes.

Keep in mind that there may be fees associated with changing your booking. These fees vary depending on the airline or travel agent and the type of fare that you booked.
    """.strip(),
    file_path="question_6.txt",
)

In [11]:
write_file(
    question="I enter the wrong email address",
    answer="""
    Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.

    If you can't remember why you booked with, you can check your credit card statement for a company name.

    The search box below can help you find the contact details for the travel provider you booked with.
    """.strip(),
    file_path="question_7.txt",
)

In [12]:
write_file(
    question="Luggage",
    answer="""
    Depending on the flight provider, the rules, conditions and prices for luggage (including sports equipment) do vary.
    It's always a good idea to check with the airline or travel agent directly (and you should be shown the options when you make your booking).
    """.strip(),
    file_path="question_8.txt",
)

In [13]:
write_file(
    question="Changes, cancellation and refunds",
    answer="""

The process for changing, canceling, and getting refunds on flights varies depending on the airline and the type of fare that you booked. However, there are some general guidelines that you can follow.
    """.strip(),
    file_path="question_9.txt",
)

In [14]:
write_file(
    question="Why does the price sometimes change when I am redirected to a flight provider?",
    answer="""
There are a few reasons why the price of a flight might change when you are redirected to a flight provider's website:

Exchange rates. Flight prices are often listed in the currency of the country where the airline is based. If you are booking a flight from a different country, the price may change depending on the current exchange rate.
Taxes and fees. Airlines and travel agents may charge different taxes and fees, so the total price of your flight may vary depending on where you book it.
Seat availability. Flight prices are often based on demand, so the price of a flight may change if the number of seats available changes.
Special offers. Flight providers may offer special discounts and promotions that are not available on Skyscanner. This is why the price of a flight may be lower on the flight provider's website.
    """.strip(),
    file_path="question_10.txt",
)

In [88]:
write_file(
    question="What is the Optimist NFT?",
    answer="""
    The Optimist NFT is a fully customizable digital avatar that allows you to showcase your personality and build your onchain identity. Powered by digital certificates called attestations, the Optimist NFT is one of the first applications built on top of the AttestationStation, an attestation smart contract on Optimism. The Optimist NFT is a fully customizable digital avatar that allows you to showcase your personality and build your onchain identity. With the Optimist NFT, you have the power to create a one-of-a-kind avatar that represents your unique personality and interests. The vision is that this NFT represents you as an individual Optimist, and evolves to reflect your identity and reputation across the city as you interact with different projects and protocols.
    Note: This is the page for users. If you are an application developer who'd like to learn more about the AttestationStation, see here.
    """.strip(),
file_path="question_11.txt",
)

# Model

In [15]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [16]:
DEVICE

'cuda:0'

In [17]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)
generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [18]:
# model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
# model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
# model = AutoGPTQForCausalLM.from_quantized(
#         model_name_or_path,
#         model_basename=model_basename,
#         use_safetensors=True,
#         trust_remote_code=True,
#         device=DEVICE,
#     )
# generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [19]:
question = (
    "which programming language is more suitable for a beginner: Python or Javascript?"
)
prompt = f"""
###Instruction: {question}
###Response:
""".strip()

In [20]:
print(prompt)

###Instruction: which programming language is more suitable for a beginner: Python or Javascript?
###Response:


In [22]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(inputs = input_ids, temperature=0.7,max_new_tokens=512)

CPU times: user 4.35 s, sys: 618 ms, total: 4.97 s
Wall time: 9.11 s


In [23]:
print(tokenizer.decode(output[0]))

<s> ###Instruction: which programming language is more suitable for a beginner: Python or Javascript?
###Response:Python is generally considered more suitable for beginners due to its readability and simplicity compared to Javascript.</s>


In [24]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [25]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)

In [26]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Peg

In [29]:
llm = HuggingFacePipeline(pipeline=pipe)

In [30]:
response = llm(prompt)

Python is generally considered to be more suitable for beginners as it has simpler syntax and fewer rules compared to JavaScript.


# **Embed Documents**

In [34]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs={"device": DEVICE},
)

In [35]:
loader = DirectoryLoader("./skyscanner/", glob="**/*txt")
documents = loader.load()
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


10

In [36]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [37]:
texts[4]

Document(page_content="Q: How do I search for flights on skyscanner? A: Skyscanner helps you find the best options for flights on a specific data, or an any day in a given month or even year. For tips on how bext to search, please head over to over to our search tip If you're looking for inspiration for your next trip, why not try our everywhere, features. Or, if you want to hang out and ensure best price, you can set up price alers to let", metadata={'source': 'skyscanner/question_1.txt'})

In [40]:
db = Chroma.from_documents(texts, embeddings)

In [41]:
db.similarity_search("flight search")

[Document(page_content="Q: How do I search for flights on skyscanner? A: Skyscanner helps you find the best options for flights on a specific data, or an any day in a given month or even year. For tips on how bext to search, please head over to over to our search tip If you're looking for inspiration for your next trip, why not try our everywhere, features. Or, if you want to hang out and ensure best price, you can set up price alers to let", metadata={'source': 'skyscanner/question_1.txt'}),
 Document(page_content='Check your spam or junk mail folder. Search your inbox for the name of the airline or travel agent that you booked with. Try contacting the airline or travel agent directly to ask for a copy of your booking confirmation.', metadata={'source': 'skyscanner/question_4.txt'}),
 Document(page_content="Q: Luggage A: Depending on the flight provider, the rules, conditions and prices for luggage (including sports equipment) do vary. It's always a good idea to check with the airline

# Conversational chain

In [42]:
template = """
### Instruction: You're a travelling support agent that is talking to a custmoer. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. IF you don't know the answer ~ say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### input: {question}
### Response:
""".strip()

In [44]:
prompt = PromptTemplate(
    input_variables={"context", "question", "chat_history"}, template=template
)

In [45]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="answer",
    return_messages=True,
)

In [53]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True,
)

In [54]:
question = "How flight search works?"
answer = chain(question)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a custmoer. Use only the chat history and the following information
Q: How do I search for flights on skyscanner? A: Skyscanner helps you find the best options for flights on a specific data, or an any day in a given month or even year. For tips on how bext to search, please head over to over to our search tip If you're looking for inspiration for your next trip, why not try our everywhere, features. Or, if you want to hang out and ensure best price, you can set up price alers to let

Exchange rates. Flight prices are often listed in the currency of the country where the airline is based. If you are booking a flight from a different country, the price may change depending on the current exchange rate. Taxes and fees. Airlines and travel agents may charge different taxes and fees, so the total price of your flight may vary depending on wher

In [55]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [56]:
answer["source_documents"]

[Document(page_content="Q: How do I search for flights on skyscanner? A: Skyscanner helps you find the best options for flights on a specific data, or an any day in a given month or even year. For tips on how bext to search, please head over to over to our search tip If you're looking for inspiration for your next trip, why not try our everywhere, features. Or, if you want to hang out and ensure best price, you can set up price alers to let", metadata={'source': 'skyscanner/question_1.txt'}),
 Document(page_content="Exchange rates. Flight prices are often listed in the currency of the country where the airline is based. If you are booking a flight from a different country, the price may change depending on the current exchange rate. Taxes and fees. Airlines and travel agents may charge different taxes and fees, so the total price of your flight may vary depending on where you book it. Seat availability. Flight prices are often based on demand, so the price of a flight may change if the

In [60]:
question = "I bought flight tickets, but I can't find any confirmation. Where is it?"
response = chain(question)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How flight search works?
Assistant: Skyscanner helps you find the best options for flights on a specific date, or an entire month/year. To get the most accurate results, make sure your search settings are correct (e.g., departure city, dates, # of passengers). Our "everywhere" feature shows all possible destinations from your starting point, while "price alerts" keep track of fare trends & notify you when prices drop.
Human: I bought flight tickets, but I can't find any confirmation. Where is it?
Assistant: Sure! Please check your spam or junk mail folder first. If it's not there, try searching your inbox for the name of the airline or travel agent that you booked with. Contact the airline or travel agent directly to request a copy of your booking confirmation. Make su

# QA Chain with Memory

In [62]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="output_text",
    return_messages=False,
)
chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

# Support Chatbot

In [89]:
DEFAULT_TEMPLATE = """
### Instruction: You're a travelling support agent that is talking to a custmoer. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. IF you don't know the answer ~ say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### input: {question}
### Response:
""".strip()

class Chatbot:
  def __init__(
      self,
      text_pipeline: HuggingFacePipeline,
      embeddings: HuggingFaceEmbeddings,
      documents_dir: Path,
      prompt_template: str = DEFAULT_TEMPLATE,
      verbose: bool=False,
  ):
      prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template=prompt_template,
      )
      self.chain = self._create_chain(text_pipeline, prompt, verbose)
      self.db = self._embed_data(documents_dir, embeddings)

  def _create_chain(
      self,
      text_pipeline: HuggingFacePipeline,
      prompt: PromptTemplate,
      verbose: bool = False,
  ):
      memory = ConversationBufferMemory(
          memory_key="chat_history",
          human_prefix="### Input",
          ai_prefix="### Response",
          input_key="question",
          output_key="output_text",
          return_messages=False,
      )
      return load_qa_chain(
          text_pipeline,
          chain_type="stuff",
          prompt=prompt,
          memory=memory,
          verbose=verbose,
      )

  def _embed_data(
      self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
  )-> Chroma:
      loader = DirectoryLoader(documents_dir, glob="**/*txt")
      documents = loader.load()
      text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
      texts = text_splitter.split_documents(documents)
      return Chroma.from_documents(texts, embeddings)
  def __call__(self, user_input: str)->str:
    docs = self.db.similarity_search(user_input)
    return self.chain.run({"input_documents": docs, "question": user_input})

In [90]:
chatbot = Chatbot(llm, embeddings, "./skyscanner/")

In [91]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

while True:
  user_input = input("Type bye to exit\nYour prompt: ")
  if user_input.lower() in ["bye", "goodbye"]:
    break
  answer = chatbot(user_input)
  print()

Type bye to exit
Your prompt: What is the Optimist NFT?
The Optimist NFT is a fully customizable digital avatar that allows you to showcase your personality and build your onchain identity. Powered by digital certificates called attestations, the Optimist NFT is one of the first applications built on top of the AttestationStation, an attestation smart contract on Optimism.

Type bye to exit
Your prompt: Can NFT customize?
Yes, the Optimist NFT is fully customizable. You can choose from a variety of facial features, hairstyles, outfits, accessories, and even pets to create a unique representation of yourself.

Type bye to exit
Your prompt: bye
